In [1]:
def median_sentences_length(data):
    all_lengths, wt_lengths, pos_lengths = [], [], []
    for d in data:
        sentence = preprocess(d)
        pos_string = get_pos_string(d)
        all_lengths.append(len(sentence.split(' ')))
        all_lengths.append(len(pos_string.split(' ')))
        wt_lengths.append(len(sentence.split(' ')))
        pos_lengths.append(len(pos_string.split(' ')))
    
    return int(np.median(all_lengths)), int(np.median(wt_lengths)), int(np.median(pos_lengths))

# Hate Speech Detector - EN - Features extraction for SVM & Dense model

Based on [this notebook](https://github.com/t-davidson/hate-speech-and-offensive-language/blob/master/classifier/final_classifier.ipynb).

In [2]:
import os
import pandas as pd
import csv
import numpy as np
import pickle
from klepto.archives import dir_archive
import sys
import nltk
import string
import re
import fasttext
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer as VS
import syllables as sylla
from pymagnitude import *

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer

from tqdm.notebook import tqdm
%matplotlib inline

In [3]:
MODEL = 'dense'
dim = 10 if MODEL == 'svm' else 200

## Davidson et al. data loading

### Classes pre:
    0 - hate speech
    1 - offensive language
    2 - neither

In [4]:
if not os.path.exists('hsd/DavidsonEtAl/perfect_data.pkl'):
    tweets, labels = [], []
    with open('hsd/DavidsonEtAl/labeled_data.csv', 'r') as f:
        for d in tqdm(list(csv.reader(f))[1:]):
            tweets.append(d[6])  # tweet
            labels.append(d[5])  # class
    with open('hsd/DavidsonEtAl/perfect_data.pkl', 'w') as f:
        def chcl(c):
            return 0 if c=='2' else 1
        labels = list(map(chcl, labels))
        pickle.dump((tweets, labels), f)
else:
    with open('hsd/DavidsonEtAl/perfect_data.pkl', 'rb') as f:
        tweets, labels = pickle.load(f)

### Classes post:
    0 - no hate
    1 - hate speech

In [5]:
print('Tweets: {}'.format(len(tweets)))
print('Labels: {}'.format(len(labels)))

Tweets: 24783
Labels: 24783


In [6]:
list(zip(tweets[:5], labels[:5]))

[("!!! RT @mayasolovely: As a woman you shouldn't complain about cleaning up your house. &amp; as a man you should always take the trash out...",
  0),
 ('!!!!! RT @mleew17: boy dats cold...tyga dwn bad for cuffin dat hoe in the 1st place!!',
  1),
 ('!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby4life: You ever fuck a bitch and she start to cry? You be confused as shit',
  1),
 ('!!!!!!!!! RT @C_G_Anderson: @viva_based she look like a tranny', 1),
 ('!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you hear about me might be true or it might be faker than the bitch who told it to ya &#57361;',
  1)]

## Features extraction

In [7]:
stopwords=stopwords = nltk.corpus.stopwords.words("english")

other_exclusions = ["#ff", "ff", "rt"]
stopwords.extend(other_exclusions)

sentiment_analyzer = VS()

In [8]:
def word_tokenization(tweet):
    lemmatizer = WordNetLemmatizer() 
    tokens = word_tokenize(tweet)
    words = [word for word in tokens if word.isalpha()]
    stop_words = set(nltk.corpus.stopwords.words("english"))
    words = [w for w in words if not w in stop_words]
    words = [lemmatizer.lemmatize(w) for w in words]
    return words

def simple_preprocess(text_string):
    """
    Accepts a text string and replaces:
    1) urls with URLHERE
    2) lots of whitespace with one instance
    3) mentions with MENTIONHERE
    4) hashtags with HASHTAGHERE

    This allows us to get standardized counts of urls and mentions
    Without caring about specific people mentioned
    """
    space_pattern = '\s+'
    giant_url_regex = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    mention_regex = '@[\w\-]+'
    #hashtag_regex = '#[\w\-]+'
    parsed_text = re.sub(space_pattern, ' ', text_string)
    parsed_text = re.sub(giant_url_regex, '', parsed_text)
    parsed_text = re.sub(mention_regex, '', parsed_text)
    #parsed_text = re.sub(hashtag_regex, '', parsed_text)
    return parsed_text

def preprocess(text_string):
    parsed_text = simple_preprocess(text_string)
    list_words = word_tokenization(parsed_text)
    parsed_text = " ".join(list_words)
    return parsed_text

def basic_tokenize(tweet):
    tweet = " ".join(re.split(" ", tweet.lower())).strip()
    return tweet.split()

def get_pos_string(tweet):
    text = preprocess(tweet)
    tokens = basic_tokenize(text)
    tags = nltk.pos_tag(tokens)
    tag_list = [x[1] for x in tags]
    tag_str = ' '.join(tag_list)
    
    return tag_str

def pad_words(words, length):
    if len(words) >= length:
        return words[:length]
    else:
        additional = length - len(words)
        return words + ['EMPTY']*additional

def count_twitter_objs(text_string):
    """
    Accepts a text string and replaces:
    1) urls with URLHERE
    2) lots of whitespace with one instance
    3) mentions with MENTIONHERE
    4) hashtags with HASHTAGHERE

    This allows us to get standardized counts of urls and mentions
    Without caring about specific people mentioned.
    
    Returns counts of urls, mentions, and hashtags.
    """
    space_pattern = '\s+'
    giant_url_regex = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    mention_regex = '@[\w\-]+'
    hashtag_regex = '#[\w\-]+'
    parsed_text = re.sub(space_pattern, ' ', text_string)
    parsed_text = re.sub(giant_url_regex, 'URLHERE', parsed_text)
    parsed_text = re.sub(mention_regex, 'MENTIONHERE', parsed_text)
    parsed_text = re.sub(hashtag_regex, 'HASHTAGHERE', parsed_text)
    return(parsed_text.count('URLHERE'),parsed_text.count('MENTIONHERE'),parsed_text.count('HASHTAGHERE'))

def other_features(tweet):
    """This function takes a string and returns a list of features.
    These include Sentiment scores, Text and Readability scores,
    as well as Twitter specific features"""
    sentiment = sentiment_analyzer.polarity_scores(tweet)
    
    words = simple_preprocess(tweet) #Get text only
    
    syllables = sylla.estimate(words)
    num_chars = sum(len(w) for w in words)
    num_chars_total = len(tweet)
    num_terms = len(tweet.split())
    num_words = len(words.split())
    avg_syl = round(float((syllables+0.001))/float(num_words+0.001),4)
    num_unique_terms = len(set(words.split()))
    
    ###Modified FK grade, where avg words per sentence is just num words/1
    FKRA = round(float(0.39 * float(num_words)/1.0) + float(11.8 * avg_syl) - 15.59, 1)
    ##Modified FRE score, where sentence fixed to 1
    FRE = round(206.835 - 1.015*(float(num_words)/1.0) - (84.6*float(avg_syl)), 2)
    
    twitter_objs = count_twitter_objs(tweet)
    retweet = 0 if "rt" in words else 1
    features = [FKRA, FRE, syllables, avg_syl, num_chars, num_chars_total, num_terms, num_words,
                num_unique_terms, sentiment['neg'], sentiment['pos'], sentiment['neu'], sentiment['compound'],
                twitter_objs[2], twitter_objs[1],
                twitter_objs[0], retweet]
    return features

### Supervised fastText wordtokens training

In [9]:
if not os.path.exists('hsd/DavidsonEtAl/fasttext.ft'):
    with open('hsd/DavidsonEtAl/fasttext.ft', 'a') as f:
        for t, l in list(zip(tweets, labels)):
            f.write('__label__{} {}\n'.format(l, preprocess(t)))

# load fasttext model or train & save if none
if os.path.exists('hsd/DavidsonEtAl/fasttext_{}.bin'.format(MODEL)):
    ft_model = fasttext.load_model('hsd/DavidsonEtAl/fasttext_{}.bin'.format(MODEL))
else:
    ft_model = fasttext.train_supervised('hsd/DavidsonEtAl/fasttext.ft',
                                         lr=0.5, epoch=50, wordNgrams=3, dim=dim)
    ft_model.save_model('hsd/DavidsonEtAl/fasttext_{}.bin'.format(MODEL))

### Wordtoken features

In [10]:
def get_wordtoken_fts(data):

    sentences_words = []
    for d in data:
        sentence = preprocess(d)
        sentences_words.append(sentence.split(' '))
    
    opt_length = int(np.median([len(sw) for sw in sentences_words]))
    sentences_words = [pad_words(sw, opt_length) for sw in sentences_words]
    vecs = Magnitude(MagnitudeUtils.download_model('http://magnitude.plasticity.ai/fasttext/light/crawl-300d-2M.magnitude'))
    
    ft_matrices = []
    for sw in sentences_words:
        ft_matrix = []
        for w in sw:
            ft_matrix.extend(vecs.query(w))
        ft_matrices.append(ft_matrix)
    
    return ft_matrices

In [11]:
wordtoken_features = get_wordtoken_fts(tweets)

In [12]:
wordtoken_features[0]

[0.0230815,
 -0.0565691,
 -0.0426789,
 0.0396304,
 -0.0277115,
 -0.0144632,
 0.0407765,
 -0.020308,
 -0.0394012,
 -0.0249151,
 -0.0062574,
 0.0789629,
 -0.0254652,
 -0.0990646,
 0.0833179,
 -0.1000273,
 0.0281928,
 -0.0275281,
 -0.0379343,
 -0.0105666,
 -0.0104291,
 0.0174887,
 -0.0011002,
 -0.0404556,
 -0.0349316,
 -0.0316539,
 -0.0807507,
 -0.0227147,
 0.0167094,
 0.0060511,
 -0.0492573,
 -0.0292243,
 -0.0547125,
 0.0625515,
 -0.0339231,
 -0.0010773,
 0.102136,
 0.0281241,
 0.0605344,
 -0.0088704,
 -0.0781377,
 0.0135005,
 0.0641559,
 0.0101082,
 0.1337441,
 -0.0288346,
 0.0077702,
 -0.000871,
 -0.042083,
 0.017695,
 0.0482487,
 0.0458879,
 0.0750434,
 0.0010544,
 0.0342669,
 -0.0497157,
 -0.0214999,
 -0.0117356,
 -0.0022233,
 0.0494636,
 0.0559731,
 0.0164802,
 0.048753,
 -0.034542,
 0.0464151,
 -0.0982395,
 0.0021317,
 0.0747684,
 0.038026,
 -0.0090309,
 -0.0010085,
 0.0173283,
 -0.0654853,
 0.0622764,
 -0.0472631,
 0.0162052,
 -0.0827219,
 0.0127441,
 -0.0290638,
 -0.0074723,
 0.0

### Supervised fastText pos training

In [13]:
if not os.path.exists('hsd/DavidsonEtAl/fasttext_pos.ft'):
    with open('hsd/DavidsonEtAl/fasttext_pos.ft', 'a') as f:
        for t, l in list(zip(tweets, labels)):
            f.write('__label__{} {}\n'.format(l, get_pos_string(t)))

# load fasttext pos model or train & save if none
if os.path.exists('hsd/DavidsonEtAl/fasttext_pos_{}.bin'.format(MODEL)):
    ft_pos_model = fasttext.load_model('hsd/DavidsonEtAl/fasttext_pos_{}.bin'.format(MODEL))
else:
    ft_pos_model = fasttext.train_supervised('hsd/DavidsonEtAl/fasttext_pos.ft',
                                             lr=0.5, epoch=50, wordNgrams=3, dim=dim)
    ft_pos_model.save_model('hsd/DavidsonEtAl/fasttext_pos_{}.bin'.format(MODEL))

### Part of speech (PoS) features

In [14]:
def get_pos_fts(data):

    #Get POS tags for tweets and save as a string
    pos_sentences = []
    for d in data:
        pos_string = get_pos_string(d)
        pos_sentences.append(pos_string)
        
        
    pos_tags = []
    for ps in pos_sentences:
        pos_tags.append(ps.split(' '))
    
    opt_length = int(np.median([len(pt) for pt in pos_tags]))
    pos_tags = [pad_words(pt, opt_length) for pt in pos_tags]
    
    ft_vectors = []
    for pt in pos_tags:
        ft_vector = []
        for t in pt:
            ft_vector.extend(ft_pos_model[t])
        ft_vectors.append(ft_vector)
    
    return ft_vectors

In [15]:
pos_features = get_pos_fts(tweets)

In [16]:
pos_features[0]

[-0.34796107,
 0.47996023,
 -0.12287073,
 -0.13428944,
 0.46666,
 0.072573185,
 0.067752235,
 -0.25249374,
 0.6032651,
 0.739502,
 -0.2670502,
 0.04370825,
 0.46350715,
 0.47913885,
 0.11060533,
 -0.2844985,
 0.1689682,
 0.0813674,
 -0.020235986,
 -0.008172552,
 0.015399778,
 0.12039019,
 -0.09692116,
 -0.3701975,
 0.080868706,
 -0.12903248,
 -0.0017701339,
 0.0025252325,
 0.011350664,
 -0.012643427,
 0.056493964,
 -0.0077822832,
 0.048282374,
 -0.008136558,
 -0.053353928,
 0.0013242874,
 0.007520803,
 -0.068015926,
 0.021926427,
 -0.027873684,
 0.102498814,
 0.04780455,
 0.19408764,
 -0.003455832,
 0.018739851,
 0.006132572,
 -0.0001524758,
 0.0015502092,
 -0.0044467943,
 0.07419634,
 -0.09113729,
 0.13315171,
 0.015814196,
 0.04991891,
 -0.018219844,
 -0.062380724,
 0.005976204,
 0.031669967,
 -0.019439936,
 0.010442777,
 -0.014483616,
 -0.024413344,
 -0.08010244,
 -0.10750612,
 0.05148064,
 -0.0057135383,
 0.18889895,
 0.02063057,
 -0.12081058,
 -0.06818831,
 0.122896105,
 0.0051067

### Other features

In [17]:
other_features = np.array([other_features(t) for t in tweets])

In [18]:
other_features[:5]

array([[ 7.4000e+00,  8.6710e+01,  2.8000e+01,  1.1200e+00,  1.2700e+02,
         1.4000e+02,  2.5000e+01,  2.5000e+01,  2.3000e+01,  0.0000e+00,
         1.1100e-01,  8.8900e-01,  4.5630e-01,  0.0000e+00,  1.0000e+00,
         0.0000e+00,  1.0000e+00],
       [ 1.7000e+00,  1.1128e+02,  1.5000e+01,  9.3750e-01,  7.7000e+01,
         8.5000e+01,  1.6000e+01,  1.6000e+01,  1.6000e+01,  2.3700e-01,
         0.0000e+00,  7.6300e-01, -6.8760e-01,  0.0000e+00,  1.0000e+00,
         0.0000e+00,  1.0000e+00],
       [ 3.4000e+00,  1.0617e+02,  1.9000e+01,  9.5000e-01,  9.3000e+01,
         1.2000e+02,  2.1000e+01,  2.0000e+01,  1.8000e+01,  5.2200e-01,
         0.0000e+00,  4.7800e-01, -9.5500e-01,  0.0000e+00,  2.0000e+00,
         0.0000e+00,  0.0000e+00],
       [-2.1000e+00,  1.2469e+02,  7.0000e+00,  8.7500e-01,  3.8000e+01,
         6.2000e+01,  9.0000e+00,  8.0000e+00,  8.0000e+00,  0.0000e+00,
         3.1400e-01,  6.8600e-01,  5.6730e-01,  0.0000e+00,  2.0000e+00,
         0.0000e+00

In [19]:
np.array(wordtoken_features).shape

(24783, 2400)

In [20]:
np.array(pos_features).shape

(24783, 1600)

In [21]:
np.array(other_features).shape

(24783, 17)

### All features and feature names

In [22]:
#Now join them all up
features = np.concatenate([wordtoken_features, pos_features, other_features],axis=1)

In [23]:
features.shape

(24783, 4017)

## Save features & labels

In [24]:
archive = dir_archive('hsd/DavidsonEtAl/X_y_{}'.format(MODEL), {'features': features, 'labels': labels}, serialized=True)
archive.dump()
del archive